In [9]:
import numpy as np
import torch
import torch.nn as nn
import time
import joblib

In [10]:
# Load the scaler used during training
scaler = joblib.load('scaler.pkl')

# Load class labels
classes = np.load("gesture_classes.npy", allow_pickle=True)
num_classes = len(classes)

# Define the model for 10 input features (5 landmarks × 2 for x and y)
class GestureClassifier(nn.Module):
    def __init__(self, input_size=10, num_classes=4):
        super(GestureClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Initialize and load model weights
model = GestureClassifier(input_size=10, num_classes=num_classes)
model.load_state_dict(torch.load("gesture_classifier_model.pth", map_location=torch.device('cpu')))
model.eval()

GestureClassifier(
  (model): Sequential(
    (0): Linear(in_features=10, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=4, bias=True)
  )
)

In [11]:
# Dummy input (you can also scale real input using your scaler)
example_input = np.random.rand(1, 10).astype(np.float32)
example_input_scaled = scaler.transform(example_input)
input_tensor = torch.from_numpy(example_input_scaled)

# Benchmark inference time
repetitions = 1000
with torch.no_grad():
    start_time = time.time()
    for _ in range(repetitions):
        output = model(input_tensor)
    end_time = time.time()

avg_inference_time_ms = (end_time - start_time) / repetitions * 1000
print(f"Average Inference Time per Sample: {avg_inference_time_ms:.6f} ms")

Average Inference Time per Sample: 0.157190 ms


In [12]:
# Dummy input
example_input = np.random.rand(1, 10).astype(np.float32)
example_input_scaled = scaler.transform(example_input)
input_tensor = torch.from_numpy(example_input_scaled)

# 🔁 Convert to TorchScript using tracing
traced_model = torch.jit.trace(model, input_tensor)
traced_model.save("gesture_classifier_traced.pt")

# 🔁 Load the TorchScript model
optimized_model = torch.jit.load("gesture_classifier_traced.pt")
optimized_model.eval()

# ⏱️ Benchmarking the TorchScript model
repetitions = 1000
with torch.no_grad():
    start_time = time.time()
    for _ in range(repetitions):
        output = optimized_model(input_tensor)
    end_time = time.time()

avg_inference_time_ms = (end_time - start_time) / repetitions * 1000
print(f"[TorchScript] Average Inference Time per Sample: {avg_inference_time_ms:.6f} ms")

[TorchScript] Average Inference Time per Sample: 0.081992 ms


In [13]:
# 🔍 Apply dynamic quantization (only Linear layers)
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8
)

# Dummy input
example_input = np.random.rand(1, 10).astype(np.float32)
input_scaled = scaler.transform(example_input)
input_tensor = torch.from_numpy(input_scaled)

# 🔁 Convert quantized model to TorchScript
traced_quant_model = torch.jit.trace(quantized_model, input_tensor)
traced_quant_model.save("gesture_classifier_quant_traced.pt")

# 🔁 Load the quantized + traced model
optimized_model = torch.jit.load("gesture_classifier_quant_traced.pt")
optimized_model.eval()

# ⏱️ Benchmarking
repetitions = 1000
with torch.no_grad():
    start_time = time.time()
    for _ in range(repetitions):
        output = optimized_model(input_tensor)
    end_time = time.time()

avg_inference_time_ms = (end_time - start_time) / repetitions * 1000
print(f"[Quantized + TorchScript] Avg Inference Time: {avg_inference_time_ms:.6f} ms")

[Quantized + TorchScript] Avg Inference Time: 0.122899 ms


In [14]:
# ⏱️ Benchmarking
repetitions = 1000
with torch.no_grad():
    start_time = time.time()
    for _ in range(repetitions):
        output = optimized_model(input_tensor)
    end_time = time.time()

avg_inference_time_ms = (end_time - start_time) / repetitions * 1000
print(f"[Quantized + TorchScript] Avg Inference Time: {avg_inference_time_ms:.6f} ms")

[Quantized + TorchScript] Avg Inference Time: 0.157802 ms
